In [0]:
create streaming table sales_pipeline as 
select *, current_timestamp() as ingestion_date from stream read_files(
  's3://jpmctraining/raw/sales',
  format => 'csv'
);


create streaming table product_pipeline as 
select *, current_timestamp() as ingestion_date from stream read_files(
  's3://jpmctraining/raw/products',
  format => 'csv'
);


create streaming table customer_pipeline as 
select *, current_timestamp() as ingestion_date from stream read_files(
  's3://jpmctraining/raw/customers',
  format => 'csv'
);

In [0]:
create streaming table mahesh_silver.sales_silver  
(CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW) as
select distinct * except (ingestion_date, _rescued_data) from stream mahesh_bronze.sales_pipeline

In [0]:
-- Create and populate the target table.
CREATE OR REFRESH STREAMING TABLE mahesh_silver.product_silver;

APPLY CHANGES INTO
  mahesh_silver.product_silver
FROM
  stream(mahesh_bronze.product_pipeline)
KEYS
  (product_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  seqNum
COLUMNS * EXCEPT
  (operation, seqNum,_rescued_data,ingestion_date)
STORED AS
  SCD TYPE 1;

In [0]:
-- Create and populate the target table.
CREATE OR REFRESH STREAMING TABLE mahesh_silver.customer_silver;

APPLY CHANGES INTO
  mahesh_silver.customer_silver
FROM
  stream(mahesh_bronze.customer_pipeline)
KEYS
  (customer_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  sequenceNum
COLUMNS * EXCEPT
  (operation, sequenceNum,_rescued_data,ingestion_date)
STORED AS
  SCD TYPE 2;

In [0]:
create materialized view mahesh_gold.customer_active as 
select * from mahesh_silver.customer_silver where `__END_AT` is null